<a href="https://colab.research.google.com/github/Nilufayeasmin299/Covid19-twitter-data-analysis-/blob/main/Covid19_word2vec_with_LTSM_70_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/project for paper /dataset/tencities_15days_april.csv")

In [ ]:
df.head()

,twitteDate,tweet_id,created_at,text,loc,user_id,verified,CleanedText,Subjectivity,Polarity,Analysis
0,2020-04-01,1245138809454395392,2020-04-01 00:00:00+00:00,thebsimone during corona stay clean fuck free ...,CA,u282394,0,thebsimon corona stay clean fuck free manit ki...,0.733333,-0.005556,Negative
1,2020-04-01,1245138809920126983,2020-04-01 00:00:00+00:00,balance joining fight against corona turning s...,CA,u460114,0,balanc join fight corona turn shoe product med...,0.000000,0.000000,Neutral
2,2020-04-01,1245138828622471168,2020-04-01 00:00:04+00:00,tory lanez asked young thug been staying coron...,WA,u127769,0,tori lanez ask young thug stay coronafre respo...,0.400000,0.100000,Positive
3,2020-04-01,1245138837057216518,2020-04-01 00:00:06+00:00,hasanthehun donald trump already trying spin c...,FL,u457103,0,hasanthehun donald trump alreadi tri spin coro...,0.000000,0.000000,Neutral
4,2020-04-01,1245138844766416896,2020-04-01 00:00:08+00:00,ghastly corona found into life affected family...,TX,u616396,0,ghast corona found life affect famili lost ano...,0.000000,0.000000,Neutral


In [ ]:
df.columns

Index(['twitteDate', 'tweet_id', 'created_at', 'text', 'loc', 'user_id',
       'verified', 'CleanedText', 'Subjectivity', 'Polarity', 'Analysis'],
      dtype='object')

In [ ]:
data=df.drop(['twitteDate','tweet_id','created_at','text','loc','user_id','verified','Subjectivity','Polarity'],axis=1)

In [ ]:
data.head()

,CleanedText,Analysis
0,thebsimon corona stay clean fuck free manit ki...,Negative
1,balanc join fight corona turn shoe product med...,Neutral
2,tori lanez ask young thug stay coronafre respo...,Positive
3,hasanthehun donald trump alreadi tri spin coro...,Neutral
4,ghast corona found life affect famili lost ano...,Neutral


In [ ]:
data.shape

(823002, 2)

In [ ]:
#impot all necessary libraries
import gensim.models.keyedvectors as word2vec #need to use due to depreceated model
from nltk.tokenize import RegexpTokenizer

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Conv1D, Dense, Flatten, MaxPooling1D, Dropout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Creat the function to get label of the dataset
def getSentiment(Analysis):
    if Analysis== 'Negative':
        return 0
    elif Analysis == 'Positive':
        return 1
    else:
      return 2
data['Sentiment']=data['Analysis'].apply(getSentiment)

In [ ]:
data.head()

,CleanedText,Analysis,Sentiment
0,thebsimon corona stay clean fuck free manit ki...,Negative,0
1,balanc join fight corona turn shoe product med...,Neutral,2
2,tori lanez ask young thug stay coronafre respo...,Positive,1
3,hasanthehun donald trump alreadi tri spin coro...,Neutral,2
4,ghast corona found life affect famili lost ano...,Neutral,2


In [ ]:
#Dividing the dataset into features and lables
tweets = data['CleanedText']
labels = data['Sentiment']

In [ ]:
#Lower and split the dialog
#and use regular expression to keep only letters we will use nltk Regular expression package
tkr = RegexpTokenizer('[a-zA-Z@]+')

In [ ]:
tweets_split = []

for i, line in enumerate(tweets):
    #print(line)
    tweet = str(line).lower().split()
    tweet = tkr.tokenize(str(tweet))
    tweets_split.append(tweet)

print(tweets_split[1])

['balanc', 'join', 'fight', 'corona', 'turn', 'shoe', 'product', 'medic', 'mask', 'output', 'https']


In [ ]:
w2vModel = word2vec.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/project for paper /dataset/GoogleNews-vectors-negative300.bin', binary=True, limit=500000)

In [ ]:
#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets_split)
X = tokenizer.texts_to_sequences(tweets_split)

In [ ]:
#length of tweet to consider
maxlentweet = 500
#add padding
X = pad_sequences(X, maxlen=maxlentweet)
print(X.shape)

(823002, 500)


In [ ]:
#create a embedding layer using Google pre triained word2vec (500000 words)
embedding_layer = Embedding(input_dim=w2vModel.syn0.shape[0], output_dim=w2vModel.syn0.shape[1], weights=[w2vModel.syn0], 
                            input_length=X.shape[1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [ ]:
#spliting dataset into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.30, random_state=42)

In [ ]:
X_train.shape

(576101, 500)

In [ ]:
y_train.shape

(576101,)

In [ ]:
num_labels=3
batch_size=128

In [ ]:
num_labels=3
from keras.layers import LSTM
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128))
model.add(Dropout(0.7))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 300)          150000000 
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 387       
Total params: 150,220,035
Trainable params: 150,220,035
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#fit model
model.fit(X_train, y_train, epochs=2, verbose=1, batch_size=batch_size)

Epoch 1/2
4501/4501 [==============================] - 15823s 4s/step - loss: 0.1460 - accuracy: 0.9480
Epoch 2/2
4501/4501 [==============================] - 15645s 3s/step - loss: 0.0088 - accuracy: 0.9975


In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn import metrics
score = metrics.log_loss(y_test, y_pred)
print("Log loss score: {}".format(score))

Log loss score: 0.008081444840340152


In [ ]:
import numpy as np 
y_pred= np.argmax(y_pred,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     43531
           1       1.00      0.99      1.00     50604
           2       1.00      1.00      1.00    152766

    accuracy                           1.00    246901
   macro avg       1.00      1.00      1.00    246901
weighted avg       1.00      1.00      1.00    246901

